#  The CrewAI Jupyter Notebook (eHjerteRehab.ipynb)


A.L. 2025-05-24 (with Gemini 2.5 Pro Preview and DeepResearch)

Using the `ehjerterehab_crew_env` defined in environment.yml 

The Jupyter Notebook, developed within the Cursor.ai IDE, would be structured with alternating code cells and Markdown cells for annotations. Below is a description of its key components:

(Markdown Cell - Annotation)

Title: eHjerteRehab: Simplified Patient Data Triage with CrewAI
Introduction: Briefly explain the purpose of the notebook – to demonstrate a basic multi-agent workflow for analyzing simulated patient data using CrewAI. Mention the agents and their roles.
(Code Cell - Setup)

In [1]:
# Import necessary libraries
from crewai import Agent, Task, Crew, Process
from crewai_tools import CodeInterpreterTool # Or other relevant tools like CSVSearchTool
from dotenv import load_dotenv
import os

# Load environment variables (for API keys)
load_dotenv()

# Configure the LLM (Example using OpenAI)
# Ensure OPENAI_API_KEY is set in your.env file or environment
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
# For Google Gemini (ensure GOOGLE_API_KEY is set):
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest",
                             verbose=True,
                             temperature=0.2,
                             google_api_key=os.getenv("GOOGLE_API_KEY"))

# (Optional) Instantiate any tools if needed globally
# e.g., code_interpreter_tool = CodeInterpreterTool(llm=llm)

ModuleNotFoundError: No module named 'langchain_google_genai'

(Markdown Cell - Annotation)

Explain the imports: Agent, Task, Crew, Process are core CrewAI components. CodeInterpreterTool allows agents to execute Python code. dotenv is for managing API keys.   
Explain LLM configuration: CrewAI agents need an LLM for reasoning. Show an example (OpenAI or Gemini). Emphasize setting API keys securely via environment variables.   


(Markdown Cell - Annotation)

Simulated Patient Data: Describe the structure of the input data for this demo.

In [ ]:
# Simulated patient data for the demo
patient_data_input =

# Convert to a string format that can be easily passed to the first agent
patient_data_summary_str = "\n".join([str(p) for p in patient_data_input])

(Markdown Cell - Annotation)

Explain that this is simplified data for demonstration. In a real system, data would come from databases or APIs.

(Markdown Cell - Annotation)

Agent Definitions: Detail the role, goal, and backstory for each agent.
(Code Cell - Agent Definitions)

In [ ]:
# Define the Vital Signs Analyzer Agent
vital_signs_analyzer = Agent(
    role='Vital Signs Monitoring Specialist',
    goal='Analyze patient vital signs data (resting HR, steps) to identify and flag any readings that are concerning. Resting HR > 90 bpm is high. Total steps < 2000 is low.',
    backstory='An AI assistant dedicated to meticulously reviewing numerical health data from cardiac rehabilitation patients, trained to spot deviations that might require further attention.',
    llm=llm,
    verbose=True,
    allow_delegation=False
    # tools=[code_interpreter_tool] # If using CodeInterpreterTool instantiated globally
)

# Define the Symptom Checker Agent
symptom_checker = Agent(
    role='Patient Symptom Screener',
    goal="Review patient-reported textual symptoms to identify and flag mentions of critical keywords such as 'chest pain', 'dizzy', 'dizziness', 'severe discomfort', 'palpitations', 'faint'.",
    backstory='An AI assistant focused on understanding patient language, specifically trained to detect urgent symptom descriptions in cardiac patient reports.',
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# Define the Clinical Reporter Agent
clinical_reporter = Agent(
    role='Clinical Summary Generator',
    goal='Compile the findings from vital signs analysis and symptom checking into a concise Markdown report. The report should list each patient, any flagged vital signs with values, and any flagged symptoms. If no flags for a patient, state "No immediate concerns noted".',
    backstory='An AI assistant skilled in synthesizing analytical information from multiple sources and presenting it clearly and efficiently in Markdown format for healthcare professionals.',
    llm=llm,
    verbose=True,
    allow_delegation=False
)

(Markdown Cell - Annotation)

Explain each agent's configuration: role, goal, backstory shape its behavior. llm assigns the language model. verbose=True is crucial for observing the agent's thought process during development and debugging, as it reveals the internal "ReAct" (Reasoning and Acting) loop – the agent's thoughts, the action it decides upon, the input to that action, and the subsequent observation. allow_delegation=False is set for simplicity in this demo.   


(Markdown Cell - Annotation)

Task Definitions: Detail the description and expected output for each task, assigning it to the respective agent.
(Code Cell - Task Definitions)

In [ ]:
# Define the task for analyzing vital signs
vital_analysis_task = Task(
    description=(
        "Analyze the following patient data summaries for vital signs concerns:\n"
        "------\n"
        "{patient_data_input_str}\n"
        "------\n"
        "Identify patients with resting heart rate (avg_resting_hr_last_24h) > 90 bpm OR total steps (total_steps_last_24h) < 2000. "
        "For each patient, list their ID and the specific vital sign(s) that are flagged with their values."
        "If no vital signs are flagged for a patient, note that."
    ),
    expected_output=(
        "A textual summary listing each patient ID and any flagged vital signs with their values, or a note if no vital signs are flagged for that patient."
    ),
    agent=vital_signs_analyzer
)

# Define the task for checking symptoms
symptom_check_task = Task(
    description=(
        "Analyze the 'reported_symptoms_text' for each patient in the following data summaries:\n"
        "------\n"
        "{patient_data_input_str}\n"
        "------\n"
        "Identify patients who report symptoms containing keywords: 'chest pain', 'dizzy', 'dizziness', 'severe discomfort', 'palpitations', 'faint'. "
        "For each patient, list their ID and the exact flagged symptom text."
        "If no critical symptoms are reported for a patient, note that."
    ),
    expected_output=(
        "A textual summary listing each patient ID and any reported critical symptoms, or a note if no critical symptoms are reported for that patient."
    ),
    agent=symptom_checker
)

# Define the task for generating the report
reporting_task = Task(
    description=(
        "Combine the vital signs analysis and symptom check analysis into a single, concise Markdown report. "
        "The report should have a section for each patient. "
        "For each patient, clearly state their ID, then list any flagged vital signs (with values) from the vital signs analysis, "
        "and any flagged symptoms from the symptom check analysis. "
        "If a patient has no flags from either analysis, state 'No immediate concerns noted for this patient.' "
        "Ensure the final output is ONLY the Markdown report, ready for display."
    ),
    expected_output=(
        "A comprehensive Markdown report summarizing all findings for each patient. "
        "Example for one patient:\n"
        "### Patient ID: P00X\n"
        "- Flagged Vital Signs: Resting HR: 95 bpm, Total Steps: 1500\n"
        "- Flagged Symptoms: Reported 'chest pain during activity'.\n"
        "(If no flags: ### Patient ID: P00Y\n - No immediate concerns noted for this patient.)"
    ),
    agent=clinical_reporter,
    context=[vital_analysis_task, symptom_check_task] # Depends on the output of the previous two tasks
)

(Markdown Cell - Annotation)

Explain task parameters: description provides instructions to the agent. expected_output guides the agent on the desired format of its result. agent links the task to an agent. context specifies that reporting_task uses outputs from vital_analysis_task and symptom_check_task.   


(Markdown Cell - Annotation)

Crew Instantiation and Kickoff: Show how to assemble the crew and run it.
(Code Cell - Crew Execution)

In [ ]:
# Instantiate the crew with a sequential process
ehjerterehab_crew = Crew(
    agents=[vital_signs_analyzer, symptom_checker, clinical_reporter],
    tasks=[vital_analysis_task, symptom_check_task, reporting_task],
    process=Process.sequential,
    verbose=2 # Use 2 for detailed crew-level logging
)

# Kick off the crew's work
# The input dictionary key 'patient_data_input_str' must match the placeholder in the task descriptions
results = ehjerterehab_crew.kickoff(inputs={'patient_data_input_str': patient_data_summary_str})

# Print the final result from the crew
print("\n\n--- Final Report ---")
print(results)

(Markdown Cell - Annotation)

Explain crew creation: agents and tasks lists define the crew members and their work. process=Process.sequential means tasks run in order. verbose=2 provides detailed logs of the crew's execution flow.   
Explain kickoff(): This starts the crew's execution. The inputs dictionary provides initial data to the tasks that have matching placeholders (e.g., {patient_data_input_str}).   
Describe the expected output: A Markdown formatted report.

4.5. Interpreting the Output and Next Steps
The output from this demo crew will be a Markdown-formatted string containing the clinical summary report. It should list each patient from the simulated data, highlighting any flagged vital signs (e.g., "Patient P002: Flagged Vital Signs: Resting HR: 98 bpm, Total Steps: 1800") and any critical reported symptoms (e.g., "Patient P002: Flagged Symptoms: Woke up with some chest pain, but it passed. Feeling a bit dizzy today."). Patients with no flags would be noted as having "No immediate concerns."

This simple prototype demonstrates the fundamental capability of CrewAI to orchestrate multiple AI agents to perform a multi-step analysis and reporting task relevant to eHjerteRehab.

Next Steps for Expansion:
This basic prototype can be significantly expanded to move closer to the comprehensive eHjerteRehab agent system envisioned:

Real Data Integration: Connect agents to actual data sources (databases, APIs for wearables, EHRs) using appropriate crewai_tools (e.g., DatabaseTool, custom API tools) instead of simulated data.
Sophisticated Tools: Equip agents with more advanced tools. For instance, the Risk Assessment Agent could use the CodeInterpreterTool to run complex ML models for risk scoring, or a custom tool that queries a medical knowledge graph. The SymptomCheckerAgent could use tools leveraging more advanced NLP models for nuanced symptom understanding rather than simple keyword matching.
More Specialized Agents: Introduce additional agents as outlined in Section 1.3 (e.g., a PersonalizedContentAgent that uses the report from the ClinicalReporterAgent to suggest interventions).
Hierarchical Process: For more complex patient management scenarios, implement a Process.hierarchical model where a "ClinicalManagerAgent" coordinates the other agents, assigns tasks dynamically, and reviews outputs before final action.
Robust Error Handling: Implement comprehensive error handling within agent tasks and tool interactions to manage API failures, data inconsistencies, or unexpected LLM outputs.
State Management: For long-running interactions or more complex patient journeys, incorporate more sophisticated state management to maintain context across multiple crew executions or patient interactions.
Integration with MLOps: If agents utilize ML models (e.g., for risk prediction), integrate the system with MLOps pipelines for model versioning, monitoring for performance degradation or data drift, and facilitating retraining cycles. This is critical for maintaining the accuracy and reliability of AI-driven insights in a production healthcare environment. Moving from a Jupyter notebook demonstration to a production-grade eHjerteRehab system requires careful consideration of these operational aspects, which are beyond the scope of a simple kickoff() but are essential for the technical manager to plan for in the actual deployment of the eHjerteRehab project.

Conclusion
Recap of AI's Transformative Impact on eHjerteRehab
The integration of Artificial Intelligence holds the promise of fundamentally transforming cardiac eRehabilitation platforms like eHjerteRehab. As explored, AI agents, orchestrated by frameworks such as CrewAI, can create a collaborative intelligence layer capable of automating complex monitoring, analysis, and communication tasks. Advanced data analysis, powered by machine learning techniques for pattern recognition in physiological and behavioral data, and natural language processing for understanding patient narratives, can unlock deeper, more nuanced insights into each patient's unique condition and progress. Furthermore, AI-driven communication tools, including intelligent clinician dashboards and personalized patient feedback systems, can make the rehabilitation journey more efficient for clinicians and more engaging and supportive for patients. Collectively, these AI methodologies can shift eHjerteRehab from a static content delivery platform to a dynamic, adaptive, and proactive system that offers truly personalized care, enhances patient adherence, and ultimately aims to improve clinical outcomes.

Future Outlook and Recommendations for AI Integration
The journey towards a fully AI-integrated eHjerteRehab platform should be approached strategically and incrementally.

Phased Implementation: It is advisable to begin with foundational AI capabilities. This could involve establishing robust data collection pipelines for wearables and PROs, followed by the development of initial AI agents for data monitoring and basic risk flagging (similar to the prototype). Subsequently, more sophisticated analytical models (ML for predictive risk, NLP for feedback analysis) and more autonomous agent functionalities (e.g., personalized content suggestion, hierarchical task management) can be progressively layered into the system.
Ethical Considerations and Data Governance: Throughout the development and deployment lifecycle, paramount importance must be given to ethical considerations, patient data privacy, and security. Compliance with healthcare data regulations (e.g., GDPR, HIPAA) is non-negotiable. Transparent policies regarding data usage and AI decision-making processes should be established.
Human-in-the-Loop (HITL) as a Core Principle: Especially for clinical decision support and patient communication involving medical advice, the HITL principle must be embedded. AI should augment, not replace, human clinicians. Mechanisms for clinician review, override, and validation of AI-generated insights or recommendations are crucial for ensuring patient safety and maintaining trust in the system.
Continuous Learning and Adaptation: The field of AI is rapidly evolving. The eHjerteRehab platform should be designed with adaptability in mind, allowing for the integration of new AI techniques, tools, and models as they mature and demonstrate value. This includes establishing processes for ongoing monitoring of AI model performance, retraining as necessary, and gathering feedback from both clinicians and patients to iteratively refine the AI components.
Focus on Explainability (XAI): To foster clinician adoption and trust, efforts should be made to incorporate XAI techniques, particularly for AI components involved in risk assessment or clinical recommendations. Understanding the "why" behind an AI's output is critical in healthcare.
By embracing these principles, eHjerteRehab can harness the transformative power of AI to deliver a new generation of cardiac eRehabilitation that is more personalized, proactive, efficient, and ultimately, more effective in supporting patients on their journey to recovery and long-term cardiovascular health.

In [ ]:

cloud.google.com
What are AI agents? Definition, examples, and types | Google Cloud
Opens in a new window

github.com
stephenc222/example-crewai: Example project demonstrating CrewAI - GitHub
Opens in a new window

docs.crewai.com
Tasks - CrewAI
Opens in a new window

aimasterclass.com
What is Collaborative Multi-Agent Systems? - AI Master Class
Opens in a new window

docs.crewai.com
Crafting Effective Agents - CrewAI
Opens in a new window

github.com
crewAI-examples/prep-for-a-meeting/main.py at main - GitHub
Opens in a new window

news-medical.net
Mayo Clinic develops AI tool to visualize complex biological data - News-Medical.net
Opens in a new window

quadratichq.com
The Importance of AI Data Analysis in Healthcare - Quadratic
Opens in a new window

thoughtful.ai
How AI-Powered Solutions Enhance Patient Experience - Thoughtful AI
Opens in a new window

pinc-ai.com
Clinical Decision Support Analytics - Stanson Health - PINC AI
Opens in a new window

lumenalta.com
9 examples of natural language processing in healthcare | NLP in healthcare - Lumenalta
Opens in a new window

eithealth.eu
Machine learning in healthcare: Uses, benefits and pioneers in the field - EIT Health
Opens in a new window

relias.com
AI in Patient Experience: Top 4 AI Use Cases in Healthcare | Relias
Opens in a new window

akira.ai
How AI Agents are Transforming Rehabilitation Monitoring in ...
Opens in a new window

digiqt.com
Is Your Clinic Ready for AI Agents in Physical Therapy? | Digiqt Blog
Opens in a new window

getondata.com
AI-powered dashboards for healthcare: The modern medical edge
Opens in a new window

reddit.com
What programs and software do I need to get started? : r/crewai
Opens in a new window

foreseemed.com
Machine Learning in Healthcare: Guide to Applications & Benefits
Opens in a new window

pmc.ncbi.nlm.nih.gov
Natural Language Processing to Extract Meaningful Information from ...
Opens in a new window

community.crewai.com
Suggested cursorrules template for working with CrewAI - General ...
Opens in a new window

cohorte.co
The Friendly Developer's Guide to CrewAI for Support Bots & Workflow Automation
Opens in a new window

basishealth.io
Personalized Health Dashboards: Design Guide & Best Practices | Basis Blog
Opens in a new window

mdpi.com
Deep Learning for Predicting Rehabilitation Success: Advancing Clinical and Patient-Reported Outcome Modeling - MDPI
Opens in a new window

app.uxcel.com
Dashboard Design for Health Platform Brief - Uxcel
Opens in a new window

researchgate.net
Machine Learning-Based Prediction Models for Healthcare Outcomes in Patients Participating in Cardiac Rehabilitation: A Systematic Review | Request PDF - ResearchGate
Opens in a new window

alejandro-ao.com
Crew AI Crash Course (Step by Step) - Alejandro AO
Opens in a new window

inspireresearch.com
Inspire Launches AI-Enabled Patient Insights | Transforming Life Sciences Research
Opens in a new window

okoone.com
What generative AI means for the future of healthcare | Okoone
Opens in a new window

thoughtspot.com
The Human-in-the-Loop Approach: Bridging AI & Human Expertise - ThoughtSpot
Opens in a new window

1000minds.com
What is Human-in-the-loop (HITL) in AI-assisted decision-making? - 1000minds
Opens in a new window

github.com
generative-ai/gemini/agent-engine ... - GitHub
Opens in a new window

docs.crewai.com
Quickstart - CrewAI
Opens in a new window

ai.google.dev
Customer Support Analysis with Gemini 2.5 Pro and CrewAI
Opens in a new window

mdpi.com
Comparative Analysis of Machine Learning Techniques for Heart Rate Prediction Employing Wearable Sensor Data - MDPI
Opens in a new window

ascopubs.org
Using Machine Learning to Predict Unplanned Hospital Utilization and Chemotherapy Management From Patient-Reported Outcome Measures | JCO Clinical Cancer Informatics - ASCO Publications
Opens in a new window

researchgate.net
Comparative Analysis of Machine Learning Techniques for Heart Rate Prediction Employing Wearable Sensor Data - ResearchGate
Opens in a new window

ibm.com
What is crewAI? - IBM
Opens in a new window

docs.crewai.com
Processes - CrewAI
Opens in a new window

pmc.ncbi.nlm.nih.gov
Population-Based Applications and Analytics Using Patient-Reported Outcome Measures - PMC - PubMed Central
Opens in a new window

docs.crewai.com
Agents - CrewAI
Opens in a new window

docs.crewai.com
Build Your First Crew - CrewAI
Opens in a new window

researchgate.net
(PDF) Creating a Comprehensive AI-Driven Risk Stratification Tool for High-Risk Heart Failure Populations - ResearchGate
Opens in a new window

arkenea.com
3 Compelling Cases of AI-Driven Patient Risk Stratification Tools - Arkenea
Opens in a new window

beyondkey.com
Healthcare Dashboard Examples | Beyond Key
Opens in a new window

github.com
alexfazio/crewAI-quickstart: A collection of notebooks ... - GitHub
Opens in a new window

aws.amazon.com
Build agentic systems with CrewAI and Amazon Bedrock | AWS ...
Opens in a new window

uibakery.io
10 Free Healthcare Dashboard Templates for Medical Data Management | UI Bakery Blog
Opens in a new window

upsolve.ai
10 Healthcare Dashboard Examples + Key Metrics To Track in 2025 - Upsolve AI
Opens in a new window

jmai.amegroups.org
Development and validation of a data visualization dashboard for automatic pain assessment and artificial intelligence analyses in cancer patients
Opens in a new window

thebricks.com
Clinical and Psychosocial Profile Dashboard Template - Bricks
Opens in a new window

Sources read but not used in the report
